<a href="https://colab.research.google.com/github/abosedealli/Stockprediction/blob/main/AnotherTwoyearsNexflixLSTM%2030days%20forcast%20-short-term.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install pandas
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from keras.layers import *
from keras.callbacks import EarlyStopping 




# load your dataset
data= pd.read_csv("/content/nexflix.csv")
data.shape
netflix= data.iloc[:int(0.8*len(data))]
print(netflix.shape)
netflix.info()
data.drop(["Date"],axis=1,inplace=True)
testdata=  data.iloc[int(0.8*len(data)):]
testdata.info()




testdata=  data.iloc[int(0.8*len(data)):]
testdata.info()


testdata.head()
#testdf.drop(["Date"],axis=1,inplace=True)
scalar = MinMaxScaler()
Netflix_scaled = scalar.fit_transform(data)
print(Netflix_scaled.shape)
Netflix_scaled

X_train = []
y_train = []
for i in range(60,netflix.shape[0]):
    X_train.append(Netflix_scaled[i-60: i])
    y_train.append(Netflix_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

model = Sequential()
model.add(LSTM(50, return_sequences=True,  activation = 'relu',input_shape=(X_train.shape[1], 6)))
model.add(LSTM(50))
model.add(Dense(1))
model.add(Dropout(0.2))

model.compile(optimizer='adam', loss='mean_squared_error')

# fit the model to the training data
model.fit(X_train, y_train,batch_size=32)


past_60 = data.tail(60)

dt = past_60.append(testdata, ignore_index = True)

     

inputs = scalar.fit_transform(dt)
print(inputs.shape)
inputs





X_test = []
y_test = []

for i in range(60, inputs.shape[0]):
    X_test.append(inputs[i-60:i])
    y_test.append(inputs[i, 0])
    
X_test, y_test = np.array(X_test), np.array(y_test).reshape(-1,1)
X_test.shape, y_test.shape

y_pred = model.predict(X_test)

node = pd.isnull(y_pred).any(1).nonzero()[0]

y_pred = np.delete(y_pred,node)
y_test = np.delete(y_test,node)
     

ans = pd.DataFrame()
ans['y_pred'] = y_pred
ans['y_test'] = y_test    
     
ans = ans.dropna()
scale = 1/scalar.scale_[0]
     

y_pred = y_pred*scale
y_test = y_test*scale



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
(604, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604 entries, 0 to 603
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       604 non-null    object 
 1   Open       604 non-null    float64
 2   High       604 non-null    float64
 3   Low        604 non-null    float64
 4   Close      604 non-null    float64
 5   Adj Close  604 non-null    float64
 6   Volume     604 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 33.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 604 to 755
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       152 non-null    float64
 1   High       152 non-null    float64
 2   Low        152 non-null    float64
 3   Close      152 non-null    float64
 4   Adj Close  152

In [16]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(ans.y_test, ans.y_pred, squared = False))      #MSE
     

0.22330839811564054


In [17]:
print(mean_squared_error(ans.y_test, ans.y_pred, squared = True))       

0.049866640668973415


In [18]:
from sklearn.metrics import mean_absolute_error
mae= np.mean(np.abs(ans.y_pred - y_test))
from keras.metrics.metrics import mean_absolute_percentage_error
mape = (np.mean(np.abs(ans.y_pred - y_test) /y_test))

print("MAE:", mae)
print("MAPE:", mape)

MAE: 69.986753195642
MAPE: inf
